In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
from tensorflow.keras.utils import load_img,img_to_array
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r'D:\MO3 LAP\Datasets\Fruits360\fruits-360_dataset\fruits-360\Training',
    target_size=(100, 100),  
    batch_size=100,  
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    r'D:\MO3 LAP\Datasets\Fruits360\fruits-360_dataset\fruits-360\Training',  
    target_size=(100, 100),
    batch_size=100,
    class_mode='categorical',
    subset='validation')

test_generator = test_datagen.flow_from_directory(
    r'D:\MO3 LAP\Datasets\Fruits360\fruits-360_dataset\fruits-360\Test',
    target_size=(100, 100),
    batch_size=100,
    class_mode='categorical',
    shuffle=False) 


Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [3]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(100, 100, 3), padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(Dense(131, activation='softmax'))

'''
RULES: 
   -output shape of conv = (int([input_shape - kernal_size + 2*padding]/strides)+1)
   -output shape of pool = (int([input_shape - pool_size + 2*padding]/strides)+1) 
   
   -num of params of a 2d layer = (kernal_w*kernal_h*input_channels+1)*(#kernals)
   -num of params of a 1d layer = (num_of_nuerons*input_shape)+num_of_nuerons

EXPECTED PARAMETERS: 
   -Any pooling layer is not trainable, no parameters
   -Conv2D = (5*5*3+1)*6 
   -Conv2D = (5*5*6+1)*16
   -dense =  (8464*120)+120
   -dense =  (120*84)+84
   -dense =  (84*131)+131
'''


'\nRULES: \n   -output shape of conv = (int([input_shape - kernal_size + 2*padding]/strides)+1)\n   -output shape of pool = (int([input_shape - pool_size + 2*padding]/strides)+1) \n   \n   -num of params of a 2d layer = (kernal_w*kernal_h*input_channels+1)*(#kernals)\n   -num of params of a 1d layer = (num_of_nuerons*input_shape)+num_of_nuerons\n\nEXPECTED PARAMETERS: \n   -Any pooling layer is not trainable, no parameters\n   -Conv2D = (5*5*3+1)*6 \n   -Conv2D = (5*5*6+1)*16\n   -dense =  (8464*120)+120\n   -dense =  (120*84)+84\n   -dense =  (84*131)+131\n'

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 6)       456       
                                                                 
 average_pooling2d (Average  (None, 50, 50, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 23, 23, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 8464)              0         
                                                                 
 dense (Dense)               (None, 120)               1

In [6]:
model.fit(train_generator,validation_data=validation_generator, epochs=15)

Epoch 1/15
542/542 [==============================] - 104s 189ms/step - loss: 1.2495 - accuracy: 0.8110 - val_loss: 0.5004 - val_accuracy: 0.8996
Epoch 2/15
542/542 [==============================] - 97s 179ms/step - loss: 0.0868 - accuracy: 0.9973 - val_loss: 0.3529 - val_accuracy: 0.9185
Epoch 3/15
542/542 [==============================] - 105s 193ms/step - loss: 0.0225 - accuracy: 0.9998 - val_loss: 0.3288 - val_accuracy: 0.9203
Epoch 4/15
542/542 [==============================] - 111s 205ms/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.3018 - val_accuracy: 0.9235
Epoch 5/15
542/542 [==============================] - 101s 187ms/step - loss: 0.0772 - accuracy: 0.9864 - val_loss: 0.3156 - val_accuracy: 0.9156
Epoch 6/15
542/542 [==============================] - 97s 180ms/step - loss: 0.0060 - accuracy: 0.9999 - val_loss: 0.3419 - val_accuracy: 0.9002
Epoch 7/15
542/542 [==============================] - 98s 181ms/step - loss: 0.0039 - accuracy: 0.9999 - val_loss: 0.2735 - va

In [7]:
model.evaluate(test_generator)

227/227 [==============================] - 25s 108ms/step - loss: 0.3299 - accuracy: 0.9248


[0.32989388704299927, 0.9248060584068298]

In [8]:
y_pred = np.argmax(model.predict(test_generator), axis=1)
Y_true = test_generator.classes 

227/227 [==============================] - 13s 58ms/step


In [9]:
cm = confusion_matrix(Y_true, y_pred)

'''[
for 2 classes:
                    ACTUAL
                     N  P
            PRED  N [TN FP
            ICTED P  FN TP]

for more classes :
                         ACTUAL
                      0     1     2 ... 
            PRE 0 [T0(0) F1(1) F2(2) 
            DIC 1  F0(0) T1(1) F2(2)
            TED 2  F0(0) F1(1) T2(2)]
                .
                .
                .
            ] 
            note that for multi class:
            to calc the measurs for class x, take the values you need only from row and column (x)


precision --> TP/(TP + FP)
recall --> TP/(TP + FN)
f1 --> mean of precision and recall
'''

'[\nfor 2 classes:\n                    ACTUAL\n                     N  P\n            PRED  N [TN FP\n            ICTED P  FN TP]\n\nfor more classes :\n                         ACTUAL\n                      0     1     2 ... \n            PRE 0 [T0(0) F1(1) F2(2) \n            DIC 1  F0(0) T1(1) F2(2)\n            TED 2  F0(0) F1(1) T2(2)]\n                .\n                .\n                .\n            ] \n            note that for multi class:\n            to calc the measurs for class x, take the values you need only from row and column (x)\n\n\nprecision --> TP/(TP + FP)\nrecall --> TP/(TP + FN)\nf1 --> mean of precision and recall\n'

In [10]:
classes = [name for name in os.listdir(r'D:\MO3 LAP\Datasets\Fruits360\fruits-360_dataset\fruits-360\Training') if os.path.isdir(os.path.join(r'D:\MO3 LAP\Datasets\Fruits360\fruits-360_dataset\fruits-360\Training', name))]

df_cm = pd.DataFrame(cm, index=[i for i in classes], columns=[i for i in classes])
plt.figure(figsize=(300, 100))
ax= sn.heatmap(df_cm, annot=True, fmt='g',cmap='Purples')
ax.xaxis.tick_top()
plt.ion()
plt.show()


In [11]:
report=classification_report(Y_true, y_pred)
print(report[0:7130])

              precision    recall  f1-score   support

           0       0.90      0.66      0.76       164
           1       1.00      1.00      1.00       148
           2       1.00      0.99      0.99       160
           3       0.77      0.87      0.82       164
           4       0.68      1.00      0.81       161
           5       0.98      0.99      0.98       164
           6       0.92      0.99      0.95       152
           7       0.72      0.95      0.82       164
           8       0.83      0.93      0.87       164
           9       1.00      1.00      1.00       144
          10       0.97      1.00      0.98       166
          11       0.88      0.70      0.78       164
          12       1.00      1.00      1.00       219
          13       1.00      1.00      1.00       164
          14       0.93      1.00      0.97       143
          15       0.81      1.00      0.90       166
          16       0.93      0.67      0.78       166
          17       0.80    